# Clasificador de supervivencia de pasajeros del Titanic

Este notebook presenta la construcción de un modelo de **clasificación supervisada** utilizando el clásico **dataset del Titanic**, el cual contiene información sobre los pasajeros del barco hundido en 1912.  
El **objetivo principal** es predecir la **probabilidad de supervivencia** de un pasajero a partir de atributos como su edad, sexo, clase en la que viajaba, número de familiares a bordo y puerto de embarque.

A diferencia de un flujo manual de preprocesamiento, en este trabajo se empleará un **Pipeline de scikit-learn**, el cual permite integrar en una sola estructura tanto el tratamiento de los datos (imputación de valores faltantes, codificación de variables categóricas y normalización de variables numéricas) como el modelo de clasificación.  

Se utilizará algoritmo de **clasificación binaria** - **Regresión Logística.** El rendimiento se evaluará mediante métricas como la **precisión** y el **f1-score**.

Finalmente, se ilustrará cómo aplicar el modelo entrenado a **casos individuales**, prediciendo la probabilidad de supervivencia de pasajeros específicos.  

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, f1_score, precision_score, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

## Obtención del Dataset

La descarga del dataset `titanic` se realiza desde OpenML por medio de fetch_openml. Esta función de scikit-learn permite la descarga de datasets desde la plataforma OpenML.

Para este caso, se imprimen las variables predictorias como X_titanic.

In [23]:
X_titanic, y_titanic = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
print(X_titanic.columns)

Index(['pclass', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare',
       'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')


In [24]:
X_titanic.head(5)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [28]:
# =========================
# 1) Imports
# =========================
import os
import numpy as np
import pandas as pd
import joblib

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    f1_score, accuracy_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_auc_score
)

# =========================
# 2) Configuración de columnas
# =========================
# Ajusta estos nombres a los de tu DataFrame X_titanic
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']

numeric_features = ['age', 'sibsp', 'parch', 'fare']
categorical_features = ['pclass', 'sex', 'embarked']

# =========================
# 3) Preprocesadores y Pipeline
# =========================
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),   # rellena nulos con mediana
    ('scaler', StandardScaler())                     # escala numéricas
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # rellena nulos con moda
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # one-hot encoding
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# =========================
# 4) Train / Test split
# =========================
# y_titanic debería ser binaria (0/1). Si no lo es, intenta castear:
try:
    y_titanic = y_titanic.astype(int)
except Exception:
    pass

X_train, X_test, y_train, y_test = train_test_split(
    X_titanic[features], y_titanic, test_size=0.2, random_state=42, stratify=y_titanic
)

# =========================
# 5) Entrenamiento
# =========================
clf.fit(X_train, y_train)

# =========================
# 6) Métricas con umbral por defecto (0.5)
# =========================
y_pred_05 = clf.predict(X_test)                                   # usa 0.5 internamente
y_proba   = clf.predict_proba(X_test)[:, 1]                       # prob de clase positiva (sobrevive=1)

print("=== Métricas (umbral 0.5) ===")
print("F1:", f1_score(y_test, y_pred_05))
print("Accuracy:", accuracy_score(y_test, y_pred_05))
print("Precision:", precision_score(y_test, y_pred_05))
print("Recall:", recall_score(y_test, y_pred_05))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_05))
print(classification_report(y_test, y_pred_05))

# =========================
# 7) (Opcional) Buscar umbral que maximiza F1 en X_test
#    *Para un trabajo más “purista”, separa un set de validación.
# =========================
thresholds = np.linspace(0.1, 0.9, 33)
f1s = []
for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)
    f1s.append((t, f1_score(y_test, y_pred_t)))

best_threshold, best_f1 = max(f1s, key=lambda x: x[1])

print("\n=== Búsqueda de umbral por F1 (en test) ===")
print(f"Mejor umbral: {best_threshold:.3f}  |  F1: {best_f1:.4f}")

# Métricas con el mejor umbral encontrado
y_pred_best = (y_proba >= best_threshold).astype(int)
print("\n=== Métricas con umbral óptimo (F1) ===")
print("F1:", f1_score(y_test, y_pred_best))
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print("Precision:", precision_score(y_test, y_pred_best))
print("Recall:", recall_score(y_test, y_pred_best))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_best))

# =========================
# 8) Guardar artefacto (modelo + umbral)
# =========================
os.makedirs("model", exist_ok=True)

# 1) Guarda SOLO el estimator/pipeline
joblib.dump(clf, "model/logistic_titanic_pipeline.pkl")

# 2) (Opcional) guarda metadatos por separado
meta = {
    "threshold": float(best_threshold),
    "features": features
}
joblib.dump(meta, "model/logistic_titanic_meta.pkl")
# o en JSON:
# with open("model/logistic_titanic_meta.json", "w") as f:
#     json.dump(meta, f)


=== Métricas (umbral 0.5) ===
F1: 0.7357512953367875
Accuracy: 0.8053435114503816
Precision: 0.7634408602150538
Recall: 0.71
ROC-AUC: 0.8673148148148149
Matriz de confusión:
 [[140  22]
 [ 29  71]]
              precision    recall  f1-score   support

           0       0.83      0.86      0.85       162
           1       0.76      0.71      0.74       100

    accuracy                           0.81       262
   macro avg       0.80      0.79      0.79       262
weighted avg       0.80      0.81      0.80       262


=== Búsqueda de umbral por F1 (en test) ===
Mejor umbral: 0.550  |  F1: 0.7634

=== Métricas con umbral óptimo (F1) ===
F1: 0.7634408602150538
Accuracy: 0.8320610687022901
Precision: 0.8255813953488372
Recall: 0.71
Matriz de confusión:
 [[147  15]
 [ 29  71]]


['model/logistic_titanic_meta.pkl']

In [30]:
import joblib
import pandas as pd

# 1) Cargar el pipeline (el estimator)
pipe = joblib.load("model/logistic_titanic_pipeline.pkl")

# 2) Cargar metadatos (umbral, etc.)
meta = joblib.load("model/logistic_titanic_meta.pkl")
thr = float(meta.get("threshold", 0.5))  # por si acaso

# 3) Pasajero de ejemplo (sin preprocesar: el pipeline se encarga)
ejemplo = pd.DataFrame([{
    "pclass": 1,
    "sex": "female",
    "age": 20,
    "sibsp": 0,
    "parch": 1,
    "fare": 80.0,
    "embarked": "C"
}])

# 4) Predecir probabilidad y aplicar umbral
p = pipe.predict_proba(ejemplo)[0, 1]
y_hat = int(p >= thr)

print(f"Ejemplo → Probabilidad de sobrevivir: {p:.3f}  → "
      f"{'Sobrevive' if y_hat==1 else 'No sobrevive'} (umbral={thr:.2f})")


Ejemplo → Probabilidad de sobrevivir: 0.963  → Sobrevive (umbral=0.55)
